# Extract Data from CMHC Housing Market Information Portal

### Prepare Workspace

In [1]:
# Load system libraries
import os
import time

# Load web scraping libraries
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# Import utils
from utils import *

### Create Function to Scrape Data

In [20]:
def scrape_category(head, name, data, desc_text, historic, sub_cat_type, sub_categories):
    
    # Loop through Census Metropolian Areas
    i = 0
    for cma in list(CMHC_CMA.keys()):
        i += 1
        cma_code = CMHC_CMA[cma]
        print(f'#### PROCESSING DATA FOR CMA {i}/{len(CMHC_CMA.keys())}: {cma} ####')
    
        try:
        
            # Set the download directory
            download_dir = os.path.join(os.getcwd(), 'raw', data)
            
            # Set Chrome options
            chrome_options = Options()
            chrome_options.add_experimental_option("prefs", {
                "download.default_directory": download_dir,
                "download.prompt_for_download": False,  # Disable download prompt
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            })
            
            # Start webdriver
            driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
            
            # Open website
            url = "https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart?id=7175&t=3#TableMapChart/" + cma_code
            driver.get(url)
            
            # Wait for the terms and conditions checkbox to be present
            checkbox_xpath = '//input[@id="iAccept"]'
            checkbox = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, checkbox_xpath)))
            
            # Execute JavaScript to click on the checkbox to accept terms and conditions
            driver.execute_script("arguments[0].click();", checkbox)
            
            # Wait for the "Get Started" button to be clickable
            button_xpath = '//p[@class="introOverlaygetStartedButton"]/a[@class="button"]'
            button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
            
            # Click on the "Get Started" button
            button.click()
            
            # Wait for the dropdown to be clickable
            dropdown_xpath = '//a[@class="subsection-link" and text()="' + head + '"]'
            dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, dropdown_xpath)))
            
            # Click on the dropdown
            dropdown.click()
            
            # Wait for metric link to be clickable
            link_xpath = '//a[text()="' + name + '"]'
            #link_xpath = "//a[text()='Average Rent ($)'][contains(@href, 'categoryLevel2=Rental%20Condominium%20Apartments')]"
            link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, link_xpath)))
            
            # Click on the metric link
            link.click()
            
            # Wait for the "Historical Time Periods" link to be clickable
            if historic:
                historical_link_xpath = '//a[text()="Historical Time Periods"]'
                historical_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, historical_link_xpath)))
                
                # Click on the "Historical Time Periods" link
                historical_link.click()
        
                # Pause for five seconds
                time.sleep(5)
                
            # Loop through sub-categories
            if len(sub_categories) > 0:
                for cat in sub_categories:
                
                    # Wait for the dropdown to be clickable
                    if sub_cat_type == 'dimension':
                        dropdown_xpath = '//a[@id="filterBydimension-18Link" and contains(@class, "menu-link")]'
                    elif sub_cat_type == 'dwelling':
                        dropdown_xpath = '//a[@id="filterBydwelling_type_desc_enLink" and contains(@class, "menu-link")]'
                    dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, dropdown_xpath)))
                    
                    # Click on the dropdown
                    dropdown.click()
                    
                    # Wait for the sub-category option to be clickable
                    if sub_cat_type == 'dimension':
                        dropdown_xpath = '//a[@data-key="dimension-18" and @data-value="' + cat + '"]'
                    elif sub_cat_type == 'dwelling':
                        dropdown_xpath = '//a[@data-key="dwelling_type_desc_en" and @data-value="' + cat + '"]'
                    subcat_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, subcat_option_xpath)))
                    
                    # Click on the sub-category option
                    subcat_option.click()
                    
                    # Pause for five seconds
                    time.sleep(5)
                    
                    # Wait for the "Export" button to be clickable
                    export_button_xpath = '//a[@id="exportTableLink" and contains(@class, "button secondary")]'
                    export_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_button_xpath)))
                    
                    # Click on the "Export" button
                    export_button.click()
                    
                    # Wait for the "Export to Spreadsheet (CSV)" option to be clickable
                    export_csv_xpath = '//a[@data-export-type="csv"]'
                    export_csv_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_csv_xpath)))
                    
                    # Click on the "Export to Spreadsheet (CSV)" option
                    export_csv_option.click()
                    
                    # Wait to ensure the export completes
                    time.sleep(5)
                    
                    # Get the list of files in the download directory
                    files = os.listdir(download_dir)
                    
                    # Filter out directories, if any
                    files = [file for file in files if os.path.isfile(os.path.join(download_dir, file))]
                    
                    # Sort files based on modification time (newest first)
                    sorted_files = sorted(files, key=lambda x: os.path.getmtime(os.path.join(download_dir, x)), reverse=True)
                    
                    # Assume at least one file is present
                    most_recent_filename = sorted_files[0]
                    
                    # Specify the new filename
                    if desc_text != '':
                        new_filename = cma + ' - ' + cat.replace("/", "or") + ' - ' + desc_text + '.csv'
                    else:
                        new_filename = cma + ' - ' + cat.replace("/", "or") + '.csv'
                    
                    # Create the full paths for both the original and new filenames
                    original_filepath = os.path.join(download_dir, most_recent_filename)
                    new_filepath = os.path.join(download_dir, new_filename)
                    
                    # Rename the file
                    os.rename(original_filepath, new_filepath)
                    
                    # Wait for a short time to ensure the rename operation completes
                    time.sleep(1)
                    
                # Close the browser window
                driver.quit()
        
            # Proceed if no subcategories are present
            else:
        
                # Wait for the "Export" button to be clickable
                export_button_xpath = '//a[@id="exportTableLink" and contains(@class, "button secondary")]'
                export_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_button_xpath)))
                
                # Click on the "Export" button
                export_button.click()
                
                # Wait for the "Export to Spreadsheet (CSV)" option to be clickable
                export_csv_xpath = '//a[@data-export-type="csv"]'
                export_csv_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_csv_xpath)))
                
                # Click on the "Export to Spreadsheet (CSV)" option
                export_csv_option.click()
                
                # Wait to ensure the export completes
                time.sleep(5)
                
                # Get the list of files in the download directory
                files = os.listdir(download_dir)
                
                # Filter out directories, if any
                files = [file for file in files if os.path.isfile(os.path.join(download_dir, file))]
                
                # Sort files based on modification time (newest first)
                sorted_files = sorted(files, key=lambda x: os.path.getmtime(os.path.join(download_dir, x)), reverse=True)
                
                # Assume at least one file is present
                most_recent_filename = sorted_files[0]
                
                # Specify the new filename
                if desc_text != '':
                    new_filename = cma + ' - ' + ' - ' + desc_text + '.csv'
                else:
                    new_filename = cma + ' - ' + '.csv'
                
                # Create the full paths for both the original and new filenames
                original_filepath = os.path.join(download_dir, most_recent_filename)
                new_filepath = os.path.join(download_dir, new_filename)
                
                # Rename the file
                os.rename(original_filepath, new_filepath)
                
                # Wait for a short time to ensure the rename operation completes
                time.sleep(1)
    
                # Close the browser window
                driver.quit()
    
        except Exception as e:
                print(f"Error occurred while processing {cma}: {e}")
                continue  # Skip to the next iteration of the loop

### Run Scraping Algorithm for Sub-Category Data

In [ ]:
['Structure Type', 'supply', 'structure-type', ['Owners', 'Renters', 'Total']]

In [26]:
items = [
    ['Age of Primary Household Maintainer', 'household', 'age'],
    ['Mobility of Primary Household Maintainer', 'household', 'mobility'],
    ['Household Type', 'household', 'type'],
    ['Household Size', 'household', 'size'],
    ['Immigrant Households', 'household', 'immigrant'],
    ['Households with Seniors', 'household', 'senior'],
    ['Households with Children Under 18', 'household', 'children'],
    ['Activity Limitations', 'household', 'activity-limits'],
    ['Aboriginal Households', 'household', 'aboriginal'],
    ['Shelter Costs', 'shelter', ''],
    ['Mortgages', 'household', 'mortgage'],
    ['Household Income', 'household', 'income'],
    ['Condominiums', 'household', 'condominium'],
    ['Housing Suitability', 'household', 'suitability'],
    ['Value of Owner-occupied Dwellings ($)', 'household', 'value'],
    ['Period of Construction and Condition of Dwelling', 'condition', '']]

In [28]:
for target in items:
    scrape_category(
    head = 'Population, Households and Housing Stock', 
    name = target[0],
    data = target[1],
    desc_text = target[2], 
    historic = True,
    sub_cat_type = 'dwelling', 
    sub_categories = [])

#### PROCESSING DATA FOR CMA 1/28: St. John's ####
#### PROCESSING DATA FOR CMA 2/28: Halifax ####
#### PROCESSING DATA FOR CMA 3/28: Ottawa ####
#### PROCESSING DATA FOR CMA 4/28: Québec ####
#### PROCESSING DATA FOR CMA 5/28: Sherbrooke ####
#### PROCESSING DATA FOR CMA 6/28: Trois-Rivières ####
#### PROCESSING DATA FOR CMA 7/28: Montréal ####
#### PROCESSING DATA FOR CMA 8/28: Oshawa ####
#### PROCESSING DATA FOR CMA 9/28: Toronto ####
#### PROCESSING DATA FOR CMA 10/28: Hamilton ####
#### PROCESSING DATA FOR CMA 11/28: St. Catharines-Niagara ####
#### PROCESSING DATA FOR CMA 12/28: Kitchener-Cambridge-Waterloo ####
#### PROCESSING DATA FOR CMA 13/28: Guelph ####
#### PROCESSING DATA FOR CMA 14/28: London ####
#### PROCESSING DATA FOR CMA 15/28: Windsor ####
#### PROCESSING DATA FOR CMA 16/28: Greater Sudbury ####
#### PROCESSING DATA FOR CMA 17/28: Winnipeg ####
#### PROCESSING DATA FOR CMA 18/28: Regina ####
#### PROCESSING DATA FOR CMA 19/28: Saskatoon ####
#### PROCESSING DATA F

In [25]:
# scrape_category(
#     head = 'Population, Households and Housing Stock', 
#     name = 'Age of Population',
#     data = 'household-maintainer',
#     desc_text = 'age', 
#     historic = True,
#     sub_cat_type = 'dwelling', 
#     sub_categories = [])

In [12]:
# # Identify data header
# head = 'Secondary Rental Market'

# # Identify data name 
# name = 'Estimated Number of Condominium Units'

# # Label data
# data = 'condo-units'

# # Add descriptive text
# desc_text = ''

# # Determine whether data is historic
# historic = False

# # Identify subcategory type
# sub_cat_type = 'dwelling' # ['dimension', 'dwelling']

# # Identify subcategories
# sub_categories = []

# # Loop through Census Metropolian Areas
# i = 0
# for cma in list(CMHC_CMA.keys()):
#     i += 1
#     cma_code = CMHC_CMA[cma]
#     print(f'#### PROCESSING DATA FOR CMA {i}/{len(CMHC_CMA.keys())}: {cma} ####')

#     try:
    
#         # Set the download directory
#         download_dir = os.path.join(os.getcwd(), 'raw', data)
        
#         # Set Chrome options
#         chrome_options = Options()
#         chrome_options.add_experimental_option("prefs", {
#             "download.default_directory": download_dir,
#             "download.prompt_for_download": False,  # Disable download prompt
#             "download.directory_upgrade": True,
#             "safebrowsing.enabled": True
#         })
        
#         # Start webdriver
#         driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
        
#         # Open website
#         url = "https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart?id=7175&t=3#TableMapChart/" + cma_code
#         driver.get(url)
        
#         # Wait for the terms and conditions checkbox to be present
#         checkbox_xpath = '//input[@id="iAccept"]'
#         checkbox = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, checkbox_xpath)))
        
#         # Execute JavaScript to click on the checkbox to accept terms and conditions
#         driver.execute_script("arguments[0].click();", checkbox)
        
#         # Wait for the "Get Started" button to be clickable
#         button_xpath = '//p[@class="introOverlaygetStartedButton"]/a[@class="button"]'
#         button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
        
#         # Click on the "Get Started" button
#         button.click()
        
#         # Wait for the dropdown to be clickable
#         dropdown_xpath = '//a[@class="subsection-link" and text()="' + head + '"]'
#         dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, dropdown_xpath)))
        
#         # Click on the dropdown
#         dropdown.click()
        
#         # Wait for metric link to be clickable
#         #link_xpath = '//a[text()="' + name + '"]'
#         link_xpath = "//a[text()='Average Rent ($)'][contains(@href, 'categoryLevel2=Rental%20Condominium%20Apartments')]"
#         link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, link_xpath)))
        
#         # Click on the metric link
#         link.click()
        
#         # Wait for the "Historical Time Periods" link to be clickable
#         if historic:
#             historical_link_xpath = '//a[text()="Historical Time Periods"]'
#             historical_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, historical_link_xpath)))
            
#             # Click on the "Historical Time Periods" link
#             historical_link.click()
    
#             # Pause for five seconds
#             time.sleep(5)
            
#         # Loop through sub-categories
#         if len(sub_categories) > 0:
#             for cat in sub_categories:
            
#                 # Wait for the dropdown to be clickable
#                 if sub_cat_type == 'dimension':
#                     dropdown_xpath = '//a[@id="filterBydimension-18Link" and contains(@class, "menu-link")]'
#                 elif sub_cat_type == 'dwelling':
#                     dropdown_xpath = '//a[@id="filterBydwelling_type_desc_enLink" and contains(@class, "menu-link")]'
#                 dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, dropdown_xpath)))
                
#                 # Click on the dropdown
#                 dropdown.click()
                
#                 # Wait for the sub-category option to be clickable
#                 if sub_cat_type == 'dimension':
#                     dropdown_xpath = '//a[@data-key="dimension-18" and @data-value="' + cat + '"]'
#                 elif sub_cat_type == 'dwelling':
#                     dropdown_xpath = '//a[@data-key="dwelling_type_desc_en" and @data-value="' + cat + '"]'
#                 subcat_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, subcat_option_xpath)))
                
#                 # Click on the sub-category option
#                 subcat_option.click()
                
#                 # Pause for five seconds
#                 time.sleep(5)
                
#                 # Wait for the "Export" button to be clickable
#                 export_button_xpath = '//a[@id="exportTableLink" and contains(@class, "button secondary")]'
#                 export_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_button_xpath)))
                
#                 # Click on the "Export" button
#                 export_button.click()
                
#                 # Wait for the "Export to Spreadsheet (CSV)" option to be clickable
#                 export_csv_xpath = '//a[@data-export-type="csv"]'
#                 export_csv_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_csv_xpath)))
                
#                 # Click on the "Export to Spreadsheet (CSV)" option
#                 export_csv_option.click()
                
#                 # Wait to ensure the export completes
#                 time.sleep(5)
                
#                 # Get the list of files in the download directory
#                 files = os.listdir(download_dir)
                
#                 # Filter out directories, if any
#                 files = [file for file in files if os.path.isfile(os.path.join(download_dir, file))]
                
#                 # Sort files based on modification time (newest first)
#                 sorted_files = sorted(files, key=lambda x: os.path.getmtime(os.path.join(download_dir, x)), reverse=True)
                
#                 # Assume at least one file is present
#                 most_recent_filename = sorted_files[0]
                
#                 # Specify the new filename
#                 if desc_text != '':
#                     new_filename = cma + ' - ' + cat.replace("/", "or") + ' - ' + desc_text + '.csv'
#                 else:
#                     new_filename = cma + ' - ' + cat.replace("/", "or") + '.csv'
                
#                 # Create the full paths for both the original and new filenames
#                 original_filepath = os.path.join(download_dir, most_recent_filename)
#                 new_filepath = os.path.join(download_dir, new_filename)
                
#                 # Rename the file
#                 os.rename(original_filepath, new_filepath)
                
#                 # Wait for a short time to ensure the rename operation completes
#                 time.sleep(1)
                
#             # Close the browser window
#             driver.quit()
    
#         # Proceed if no subcategories are present
#         else:
    
#             # Wait for the "Export" button to be clickable
#             export_button_xpath = '//a[@id="exportTableLink" and contains(@class, "button secondary")]'
#             export_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_button_xpath)))
            
#             # Click on the "Export" button
#             export_button.click()
            
#             # Wait for the "Export to Spreadsheet (CSV)" option to be clickable
#             export_csv_xpath = '//a[@data-export-type="csv"]'
#             export_csv_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_csv_xpath)))
            
#             # Click on the "Export to Spreadsheet (CSV)" option
#             export_csv_option.click()
            
#             # Wait to ensure the export completes
#             time.sleep(5)
            
#             # Get the list of files in the download directory
#             files = os.listdir(download_dir)
            
#             # Filter out directories, if any
#             files = [file for file in files if os.path.isfile(os.path.join(download_dir, file))]
            
#             # Sort files based on modification time (newest first)
#             sorted_files = sorted(files, key=lambda x: os.path.getmtime(os.path.join(download_dir, x)), reverse=True)
            
#             # Assume at least one file is present
#             most_recent_filename = sorted_files[0]
            
#             # Specify the new filename
#             if desc_text != '':
#                 new_filename = cma + ' - ' + ' - ' + desc_text + '.csv'
#             else:
#                 new_filename = cma + ' - ' + '.csv'
            
#             # Create the full paths for both the original and new filenames
#             original_filepath = os.path.join(download_dir, most_recent_filename)
#             new_filepath = os.path.join(download_dir, new_filename)
            
#             # Rename the file
#             os.rename(original_filepath, new_filepath)
            
#             # Wait for a short time to ensure the rename operation completes
#             time.sleep(1)

#             # Close the browser window
#             driver.quit()

#     except Exception as e:
#             print(f"Error occurred while processing {cma}: {e}")
#             continue  # Skip to the next iteration of the loop

#### PROCESSING DATA FOR CMA 1/28: St. John's ####
#### PROCESSING DATA FOR CMA 2/28: Halifax ####
#### PROCESSING DATA FOR CMA 3/28: Ottawa ####
#### PROCESSING DATA FOR CMA 4/28: Québec ####
#### PROCESSING DATA FOR CMA 5/28: Sherbrooke ####
#### PROCESSING DATA FOR CMA 6/28: Trois-Rivières ####
#### PROCESSING DATA FOR CMA 7/28: Montréal ####
#### PROCESSING DATA FOR CMA 8/28: Oshawa ####
#### PROCESSING DATA FOR CMA 9/28: Toronto ####
#### PROCESSING DATA FOR CMA 10/28: Hamilton ####
#### PROCESSING DATA FOR CMA 11/28: St. Catharines-Niagara ####
#### PROCESSING DATA FOR CMA 12/28: Kitchener-Cambridge-Waterloo ####
#### PROCESSING DATA FOR CMA 13/28: Guelph ####
#### PROCESSING DATA FOR CMA 14/28: London ####
#### PROCESSING DATA FOR CMA 15/28: Windsor ####
#### PROCESSING DATA FOR CMA 16/28: Greater Sudbury ####
#### PROCESSING DATA FOR CMA 17/28: Winnipeg ####
#### PROCESSING DATA FOR CMA 18/28: Regina ####
#### PROCESSING DATA FOR CMA 19/28: Saskatoon ####
#### PROCESSING DATA F

### Run Scraping Algorithm for No Sub Category Data

In [ ]:
# Loop through Census Metropolian Areas
i = 0
for cma in list(CMHC_CMA.keys()):
    i += 1
    cma_code = CMHC_CMA[cma]
    print(f'#### PROCESSING DATA FOR CMA {i}/{len(CMHC_CMA.keys())}: {cma} ####')
    
    # Set the download directory
    download_dir = os.path.join(os.getcwd(), 'raw', 'starts')
    
    # Set Chrome options
    chrome_options = Options()
    chrome_options.add_experimental_option("prefs", {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,  # Disable download prompt
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    })
    
    # Start webdriver
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=chrome_options)
    
    # Open website
    url = "https://www03.cmhc-schl.gc.ca/hmip-pimh/en/TableMapChart?id=7175&t=3#TableMapChart/" + cma_code
    driver.get(url)

    # Begin data extraction
    try:
    
        # Wait for the terms and conditions checkbox to be present
        checkbox_xpath = '//input[@id="iAccept"]'
        checkbox = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, checkbox_xpath)))
        
        # Execute JavaScript to click on the checkbox to accept terms and conditions
        driver.execute_script("arguments[0].click();", checkbox)
        
        # Wait for the "Get Started" button to be clickable
        button_xpath = '//p[@class="introOverlaygetStartedButton"]/a[@class="button"]'
        button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
        
        # Click on the "Get Started" button
        button.click()
        
        # Wait for the dropdown to be clickable
        dropdown_xpath = '//a[@class="subsection-link"]'
        dropdown = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, dropdown_xpath)))
        
        # Click on the dropdown
        dropdown.click()
        
        # Wait for metric link to be clickable
        link_xpath = '//a[text()="Starts (SAAR)"]'
        link = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, link_xpath)))
        
        # Click on the metric link
        link.click()
        
        # Wait for the "Export" button to be clickable
        export_button_xpath = '//a[@id="exportTableLink" and contains(@class, "button secondary")]'
        export_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_button_xpath)))
        
        # Click on the "Export" button
        export_button.click()
        
        # Wait for the "Export to Spreadsheet (CSV)" option to be clickable
        export_csv_xpath = '//a[@data-export-type="csv"]'
        export_csv_option = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, export_csv_xpath)))
        
        # Click on the "Export to Spreadsheet (CSV)" option
        export_csv_option.click()
        
        # Wait to ensure the export completes
        time.sleep(5)
        
        # Get the list of files in the download directory
        files = os.listdir(download_dir)
        
        # Filter out directories, if any
        files = [file for file in files if os.path.isfile(os.path.join(download_dir, file))]
        
        # Sort files based on modification time (newest first)
        sorted_files = sorted(files, key=lambda x: os.path.getmtime(os.path.join(download_dir, x)), reverse=True)
        
        # Assume at least one file is present
        most_recent_filename = sorted_files[0]
        
        # Specify the new filename
        new_filename = cma + ' - ' + 'SAAR.csv'
        
        # Create the full paths for both the original and new filenames
        original_filepath = os.path.join(download_dir, most_recent_filename)
        new_filepath = os.path.join(download_dir, new_filename)
        
        # Rename the file
        os.rename(original_filepath, new_filepath)
        
        # Wait for a short time to ensure the rename operation completes
        time.sleep(1)
        
        # Close the browser window
        driver.quit()

    # Move on to next CMA if some error arises
    except TimeoutException:
        print("Link not clickable. Skipping to the next CMA.")
        
        # Close the browser window
        driver.quit()
        continue